# DP 4 Drone Race
by Emily Lory and Conan Zhang

In [34]:
from sympy import * 
import numpy as np
from scipy import linalg
import matplotlib.pyplot as plt
init_printing()

In [2]:
# Get the gravity in m/s^2
from ae353_quadrotor import get_gravity
planet = "Earth"
grav = get_gravity(planet, verbose=True)

In [3]:
# Mass and moment of inertia of the bus
mass = 0.5    # The mass of the quadrotor in kg
Ixx = 0.0023  # The moment of inertia of the quadrotor in 
Iyy = 0.0023  # The mass of the quadrotor in kg
Izz = 0.0040  # The mass of the quadrotor in kg
lxy = 0.25    # The x or y distance from the quadrotor frame to the mocap markers in meters
lz = 0.046875 # The z distance from the quadrotor frame to the mocap markers in meters

In [42]:
# Define position
px_inW, py_inW, pz_inW = symbols('p_x, p_y, p_z')

# Define velocity
vx_inB, vy_inB, vz_inB = symbols('v_x, v_y, v_z')

# Define roll (phi), pitch (theta), and yaw (psi) angles
phi, theta, psi = symbols('phi, theta, psi')

# Define angular velocities
wx_inB, wy_inB, wz_inB = symbols('omega_x, omega_y, omega_z')

# Define roll, pitch, and yaw rotation matrices that describe the bus's orientation in the world frame
taux_inB, tauy_inB, tauz_inB, fz_inB = symbols('tau_x, tau_y, tau_z, f_z')

In [43]:
p_inW = Matrix([[px_inW],[py_inW], [pz_inW]])
v_inB = Matrix([[vx_inB],[vy_inB],[vz_inB]])
w_inB = Matrix([[wx_inB],[wy_inB], [wz_inB]])
Rx = Matrix([[1,0,0],[0, cos(phi), -sin(phi)],[0, sin(phi),  cos(phi)]])
Ry = Matrix([[ cos(theta), 0, sin(theta)],[0, 1,0], [-sin(theta), 0, cos(theta)]])
Rz = Matrix([[cos(psi), -sin(psi), 0],[sin(psi),  cos(psi), 0],[0,0,1]])
R_ofB_inW = Rz @ Ry @ Rx
R_ofW_inB = R_ofB_inW.T

In [44]:
# Get the net torque vector
tau_inB = Matrix([[taux_inB],
                      [tauy_inB],
                      [tauz_inB]])
tau_inB = simplify(tau_inB)

# Get the gravity force and convert to body coordiantes
grav_inW = Matrix([[0.],
                       [0.],
                       [-mass * grav]])
grav_inB = R_ofW_inB @ grav_inW

# Get the net force vector
f_inB = grav_inB + Matrix([[0.],
                               [0.],
                               [fz_inB]])
f_inB = simplify(f_inB)

In [6]:
# Assemble the moment of inertia matrix
I_inB = Matrix([[Ixx, 0.,  0.],
                    [0.,  Iyy, 0.],
                    [0.,  0.,  Izz]])

In [7]:
v_inW = R_ofB_inW @ v_inB
xyz_dot = simplify(v_inW)
print("px, py, pz rate as a function of orientation and body-fixed velocities:")
N(xyz_dot,3)

px, py, pz rate as a function of orientation and body-fixed velocities:


⎡vₓ⋅cos(ψ)⋅cos(θ) + v_y⋅(sin(φ)⋅sin(θ)⋅cos(ψ) - sin(ψ)⋅cos(φ)) + v_z⋅(sin(φ)⋅s
⎢                                                                             
⎢vₓ⋅sin(ψ)⋅cos(θ) + v_y⋅(sin(φ)⋅sin(ψ)⋅sin(θ) + cos(φ)⋅cos(ψ)) - v_z⋅(sin(φ)⋅c
⎢                                                                             
⎣                            -vₓ⋅sin(θ) + v_y⋅sin(φ)⋅cos(θ) + v_z⋅cos(φ)⋅cos(θ

in(ψ) + sin(θ)⋅cos(φ)⋅cos(ψ))⎤
                             ⎥
os(ψ) - sin(ψ)⋅sin(θ)⋅cos(φ))⎥
                             ⎥
)                            ⎦

In [8]:
v_inB_dot = (1 / mass) * (f_inB - w_inB.cross(mass * v_inB))
v_inB_dot = simplify(v_inB_dot)
print("Body-fixed velocity rates as a function of orientation, body-fixed velocities, and applied forces:")
N(v_inB_dot,3)

Body-fixed velocity rates as a function of orientation, body-fixed velocities, and applied forces:


⎡       -ω_y⋅v_z + 1.0⋅ω_z⋅v_y + 9.81⋅sin(θ)       ⎤
⎢                                                  ⎥
⎢     1.0⋅ωₓ⋅v_z - ω_z⋅vₓ - 9.81⋅sin(φ)⋅cos(θ)     ⎥
⎢                                                  ⎥
⎣2.0⋅f_z - ωₓ⋅v_y + 1.0⋅ω_y⋅vₓ - 9.81⋅cos(φ)⋅cos(θ)⎦

In [9]:
# Define the transformation that takes roll, pitch, and yaw rates to body-fixed angular rates
ex = Matrix([[1], [0], [0]])
ey = Matrix([[0], [1], [0]])
ez = Matrix([[0], [0], [1]])
M = Matrix.hstack(ex, Rx.T@ey, (Ry@Rx).T@ez)

# Invert the transformation to get a matrix that takes body-fixed angular rates to 
# roll, pitch, and yaw rates
M_inv = simplify(M.inv())

# Convert the body-fixed angular rates to roll, pitch, and yaw rates
rpy_dot = simplify(M_inv@w_inB)
print("Roll, pitch, and yaw rate as a function of orientation and body-fixed angular rates:")
N(rpy_dot,3)

Roll, pitch, and yaw rate as a function of orientation and body-fixed angular rates:


⎡ωₓ + ω_y⋅sin(φ)⋅tan(θ) + ω_z⋅cos(φ)⋅tan(θ)⎤
⎢                                          ⎥
⎢         ω_y⋅cos(φ) - ω_z⋅sin(φ)          ⎥
⎢                                          ⎥
⎢         ω_y⋅sin(φ) + ω_z⋅cos(φ)          ⎥
⎢         ───────────────────────          ⎥
⎣                  cos(θ)                  ⎦

In [10]:
# Apply Euler's equation to get the time derivative of the angular velocities of the
# quadrotor in the quadrotor frame
w_inB_dot = I_inB.inv() @ (tau_inB - w_inB.cross(I_inB@w_inB))
w_inB_dot = simplify(w_inB_dot)
print("Time derivative of the angular velocities in the body-fixed frame:")
N(w_inB_dot, 3)

Time derivative of the angular velocities in the body-fixed frame:


⎡-0.739⋅ω_y⋅ω_z + 435.0⋅τₓ⎤
⎢                         ⎥
⎢0.739⋅ωₓ⋅ω_z + 435.0⋅τ_y ⎥
⎢                         ⎥
⎣        250.0⋅τ_z        ⎦

In [11]:
f = Matrix.vstack(xyz_dot,
                      v_inB_dot,
                      rpy_dot,
                      w_inB_dot)

In [12]:
N(f,3)

⎡vₓ⋅cos(ψ)⋅cos(θ) + v_y⋅(sin(φ)⋅sin(θ)⋅cos(ψ) - sin(ψ)⋅cos(φ)) + v_z⋅(sin(φ)⋅s
⎢                                                                             
⎢vₓ⋅sin(ψ)⋅cos(θ) + v_y⋅(sin(φ)⋅sin(ψ)⋅sin(θ) + cos(φ)⋅cos(ψ)) - v_z⋅(sin(φ)⋅c
⎢                                                                             
⎢                            -vₓ⋅sin(θ) + v_y⋅sin(φ)⋅cos(θ) + v_z⋅cos(φ)⋅cos(θ
⎢                                                                             
⎢                                   -ω_y⋅v_z + 1.0⋅ω_z⋅v_y + 9.81⋅sin(θ)      
⎢                                                                             
⎢                                 1.0⋅ωₓ⋅v_z - ω_z⋅vₓ - 9.81⋅sin(φ)⋅cos(θ)    
⎢                                                                             
⎢                            2.0⋅f_z - ωₓ⋅v_y + 1.0⋅ω_y⋅vₓ - 9.81⋅cos(φ)⋅cos(θ
⎢                                                                             
⎢                                ωₓ + ω_y⋅sin(φ)⋅tan

In [13]:
# Position of markers in body frame
mark1_inB = Matrix([[lxy],
                        [0.],
                        [lz]])
mark2_inB = Matrix([[0.],
                        [lxy],
                        [lz]])
mark3_inB = Matrix([[-lxy],
                        [0.],
                        [lz]])
mark4_inB = Matrix([[0.],
                        [-lxy],
                        [lz]])

# Position of markers in world frame
mark1_inW = p_inW + R_ofB_inW @ mark1_inB
mark2_inW = p_inW + R_ofB_inW @ mark2_inB
mark3_inW = p_inW + R_ofB_inW @ mark3_inB
mark4_inW = p_inW + R_ofB_inW @ mark4_inB

# Sensor model
g = simplify(Matrix.vstack(mark1_inW, mark2_inW, mark3_inW, mark4_inW))

In [14]:
N(g,3)

⎡              pₓ + 0.0469⋅sin(φ)⋅sin(ψ) + 0.0469⋅sin(θ)⋅cos(φ)⋅cos(ψ) + 0.25⋅
⎢                                                                             
⎢              p_y - 0.0469⋅sin(φ)⋅cos(ψ) + 0.0469⋅sin(ψ)⋅sin(θ)⋅cos(φ) + 0.25
⎢                                                                             
⎢                                p_z - 0.25⋅sin(θ) + 0.0469⋅cos(φ)⋅cos(θ)     
⎢                                                                             
⎢pₓ + 0.0469⋅sin(φ)⋅sin(ψ) + 0.25⋅sin(φ)⋅sin(θ)⋅cos(ψ) - 0.25⋅sin(ψ)⋅cos(φ) + 
⎢                                                                             
⎢p_y + 0.25⋅sin(φ)⋅sin(ψ)⋅sin(θ) - 0.0469⋅sin(φ)⋅cos(ψ) + 0.0469⋅sin(ψ)⋅sin(θ)
⎢                                                                             
⎢                             p_z + 0.25⋅sin(φ)⋅cos(θ) + 0.0469⋅cos(φ)⋅cos(θ) 
⎢                                                                             
⎢              pₓ + 0.0469⋅sin(φ)⋅sin(ψ) + 0.0469⋅si

## System Linearization and State Space Form

In [15]:
# FIXME: Find an equilibrium point and linearize the system into state space form
p_x_e = p_y_e = p_z_e = v_x_e = v_y_e = v_z_e = phi_e = theta_e = psi_e = omega_x_e = omega_y_e = omega_z_e = tau_x_e = tau_y_e = tau_z_e = 0
f_z_e = 4.905

m_e = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
n_e = np.array([0, 0, 0, 4.905])

In [16]:
f_num = lambdify([px_inW, py_inW, pz_inW, vx_inB, vy_inB, vz_inB, phi, theta, psi, wx_inB, wy_inB, wz_inB, taux_inB, tauy_inB, tauz_inB, fz_inB], f)
f_num(p_x_e, p_y_e, p_z_e, v_x_e, v_y_e, v_z_e, phi_e, theta_e, psi_e, omega_x_e, omega_y_e, omega_z_e, tau_x_e, tau_y_e, tau_z_e, f_z_e)

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.]])

In [25]:
# Solving for Coefficient Matrix A
A_num = lambdify([px_inW, py_inW, pz_inW, vx_inB, vy_inB, vz_inB, phi, theta, psi, wx_inB, wy_inB, wz_inB, taux_inB, tauy_inB, tauz_inB, fz_inB], 
                 f.jacobian([px_inW, py_inW, pz_inW, vx_inB, vy_inB, vz_inB, phi, theta, psi, wx_inB, wy_inB, wz_inB]))
A = A_num(p_x_e, p_y_e, p_z_e, v_x_e, v_y_e, v_z_e, phi_e, theta_e, psi_e, omega_x_e, omega_y_e, omega_z_e, tau_x_e, tau_y_e, tau_z_e, f_z_e)
Matrix(A)

⎡0  0  0  1.0   0    0     0     0    0   0    0    0 ⎤
⎢                                                     ⎥
⎢0  0  0   0   1.0   0     0     0    0   0    0    0 ⎥
⎢                                                     ⎥
⎢0  0  0   0    0   1.0    0     0    0   0    0    0 ⎥
⎢                                                     ⎥
⎢0  0  0   0    0    0     0    9.81  0   0    0    0 ⎥
⎢                                                     ⎥
⎢0  0  0   0    0    0   -9.81   0    0   0    0    0 ⎥
⎢                                                     ⎥
⎢0  0  0   0    0    0     0     0    0   0    0    0 ⎥
⎢                                                     ⎥
⎢0  0  0   0    0    0     0     0    0  1.0   0    0 ⎥
⎢                                                     ⎥
⎢0  0  0   0    0    0     0     0    0   0   1.0   0 ⎥
⎢                                                     ⎥
⎢0  0  0   0    0    0     0     0    0   0    0   1.0⎥
⎢                                               

In [26]:
# Solving for Coefficient Matrix B
B_num = lambdify([px_inW, py_inW, pz_inW, vx_inB, vy_inB, vz_inB, phi, theta, psi, wx_inB, wy_inB, wz_inB, taux_inB, tauy_inB, tauz_inB, fz_inB], 
                 f.jacobian([taux_inB, tauy_inB, tauz_inB, fz_inB]))
B = B_num(p_x_e, p_y_e, p_z_e, v_x_e, v_y_e, v_z_e, phi_e, theta_e, psi_e, omega_x_e, omega_y_e, omega_z_e, tau_x_e, tau_y_e, tau_z_e, f_z_e)
Matrix(B)

⎡       0                 0            0     0 ⎤
⎢                                              ⎥
⎢       0                 0            0     0 ⎥
⎢                                              ⎥
⎢       0                 0            0     0 ⎥
⎢                                              ⎥
⎢       0                 0            0     0 ⎥
⎢                                              ⎥
⎢       0                 0            0     0 ⎥
⎢                                              ⎥
⎢       0                 0            0    2.0⎥
⎢                                              ⎥
⎢       0                 0            0     0 ⎥
⎢                                              ⎥
⎢       0                 0            0     0 ⎥
⎢                                              ⎥
⎢       0                 0            0     0 ⎥
⎢                                              ⎥
⎢434.782608695652         0            0     0 ⎥
⎢                                              ⎥
⎢       0          4

In [27]:
# Checking Controllability
n = A.shape[0]
Wc = B
for i in range(1, n):
    col = np.linalg.matrix_power(A, i) @ B
    Wc = np.block([Wc, col])
print(np.linalg.matrix_rank(Wc) == n)

True


In [28]:
# Solving for Coefficient Matrix C
C_num = lambdify([px_inW, py_inW, pz_inW, vx_inB, vy_inB, vz_inB, phi, theta, psi, wx_inB, wy_inB, wz_inB, taux_inB, tauy_inB, tauz_inB, fz_inB], 
                 g.jacobian([px_inW, py_inW, pz_inW, vx_inB, vy_inB, vz_inB, phi, theta, psi, wx_inB, wy_inB, wz_inB]))
C = C_num(p_x_e, p_y_e, p_z_e, v_x_e, v_y_e, v_z_e, phi_e, theta_e, psi_e, omega_x_e, omega_y_e, omega_z_e, tau_x_e, tau_y_e, tau_z_e, f_z_e)
Matrix(C)

⎡1.0   0    0   0  0  0      0      0.046875    0    0  0  0⎤
⎢                                                           ⎥
⎢ 0   1.0   0   0  0  0  -0.046875     0      0.25   0  0  0⎥
⎢                                                           ⎥
⎢ 0    0   1.0  0  0  0      0       -0.25      0    0  0  0⎥
⎢                                                           ⎥
⎢1.0   0    0   0  0  0      0      0.046875  -0.25  0  0  0⎥
⎢                                                           ⎥
⎢ 0   1.0   0   0  0  0  -0.046875     0        0    0  0  0⎥
⎢                                                           ⎥
⎢ 0    0   1.0  0  0  0    0.25        0        0    0  0  0⎥
⎢                                                           ⎥
⎢1.0   0    0   0  0  0      0      0.046875    0    0  0  0⎥
⎢                                                           ⎥
⎢ 0   1.0   0   0  0  0  -0.046875     0      -0.25  0  0  0⎥
⎢                                                           ⎥
⎢ 0    0

In [29]:
# Solving for Coefficient Matrix D
D_num = lambdify([px_inW, py_inW, pz_inW, vx_inB, vy_inB, vz_inB, phi, theta, psi, wx_inB, wy_inB, wz_inB, taux_inB, tauy_inB, tauz_inB, fz_inB], 
                 g.jacobian([taux_inB, tauy_inB, tauz_inB, fz_inB]))
D = D_num(p_x_e, p_y_e, p_z_e, v_x_e, v_y_e, v_z_e, phi_e, theta_e, psi_e, omega_x_e, omega_y_e, omega_z_e, tau_x_e, tau_y_e, tau_z_e, f_z_e)
Matrix(D)

⎡0  0  0  0⎤
⎢          ⎥
⎢0  0  0  0⎥
⎢          ⎥
⎢0  0  0  0⎥
⎢          ⎥
⎢0  0  0  0⎥
⎢          ⎥
⎢0  0  0  0⎥
⎢          ⎥
⎢0  0  0  0⎥
⎢          ⎥
⎢0  0  0  0⎥
⎢          ⎥
⎢0  0  0  0⎥
⎢          ⎥
⎢0  0  0  0⎥
⎢          ⎥
⎢0  0  0  0⎥
⎢          ⎥
⎢0  0  0  0⎥
⎢          ⎥
⎣0  0  0  0⎦

In [30]:
# Checking for Observability
n = np.shape(A)[0]
Wo = C
for i in range(1, n):
    col = C@np.linalg.matrix_power(A, i)
    Wo = np.block([[Wo], [col]])
print(np.linalg.matrix_rank(Wo) == n)

True


In [31]:
def lqr(A,B,Q,R):
    P = linalg.solve_continuous_are(A, B, Q, R)
    K = linalg.inv(R) @  B.T @ P
    return K

In [47]:
Q = np.diag([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
R = np.diag([1,1,1,1])
K = lqr(A,B,Q,R)
Matrix(K)

⎡ 2.9374804585511e-15    -0.999999999999999    1.14981763858077e-16   5.089123
⎢                                                                             
⎢  0.999999999999998    6.17317562947598e-15   -1.24776593082889e-15    1.4538
⎢                                                                             
⎢2.43299909574285e-15   -1.61426080424604e-15  -1.05013547673617e-15  5.042134
⎢                                                                             
⎣-6.60357872255238e-16  -1.16999131698943e-16           1.0           -4.58492

42858674e-16     -1.45381723139033    4.25482436275894e-16     5.4621321799200
                                                                              
1723139033     2.30716928665417e-15   -7.94479603212576e-16  -3.55489580983745
                                                                              
46327273e-15   -2.56109288364565e-15  -2.7489104370163e-16   7.37503249186675e
                                                   

In [48]:
Ro = np.diag([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
Qo = np.diag([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
L = lqr(A.T,C.T,linalg.inv(Ro),linalg.inv(Qo)).T

In [49]:
print(linalg.eigvals(A-B@K))
print(linalg.eigvals(A-L@C))

[-434.78145927+0.j         -434.78145927+0.j
 -249.99799996+0.j           -2.21469792+2.21475479j
   -2.21469792-2.21475479j   -2.21469792+2.21475479j
   -2.21469792-2.21475479j   -1.41421355+0.j
   -1.41421358+0.j           -1.00000003+0.j
   -1.00000003+0.j           -1.000008  +0.j        ]
[-1.40682801+2.17021669e+00j -1.40682801-2.17021669e+00j
 -1.40682801+2.17021669e+00j -1.40682801-2.17021669e+00j
 -2.93130819+0.00000000e+00j -2.93130819+0.00000000e+00j
 -0.55901699+4.33012702e-01j -0.55901699-4.33012702e-01j
 -1.00128528+0.00000000e+00j -1.00128528+0.00000000e+00j
 -1.41421356+4.78610025e-08j -1.41421356-4.78610025e-08j]


## Gain Selection and Controller Design

In [20]:
# FIXME: Select a set of control and observer gains. Ensure the resultant system is stable. Implement your controller below.

In [ ]:
# Create the Controller class
class Controller():
    def __init__(self):
        """
        Initializes the controller. You should initialize any member variables that
        the controller will use to their starting values. This function is called
        when a new instance of the Controller class is created.
    
        Parameters
        ----------
        None.
    
        Returns
        -------
        None.
        """
        # Fixme: Implement your initialization function here
        pass

    
    def reset(self):
        """
        Resets the controller to an initial state. You should reset any variables
        that track states, times, etc. in this function. This function will be 
        called just before the simulation starts running and any time the simulation
        is reset.
    
        Parameters
        ----------
        None.
    
        Returns
        -------
        None.
        """
        # Fixme: Implement your reset function here
        pass
    
    
    def run(self, **kwargs):
        """
        Runs the controller. Data is passed from the simulation environment to the
        controller via kwargs, the controller calculates the inputs to the system,
        and then returns them via a return statement
    
        Parameters
        ----------
        **kwargs : Dictionary
            A dictionary of data passed from the simulation environment to the 
            controller. Specific data is extracted from kwargs via the statement:
            kwargs["KEYWORD"], where "KEYWORD" is a key to the dictionary. The keys
            of kwargs for this project are as follows:

            kwargs["dt"] : float
                The simulation time step in seconds.

            kwargs["mocap_1"] : 3Vector
                The noisy measured position of mocap marker 1 (+x) in meters.
            
            kwargs["mocap_2"] : 3Vector
                The noisy measured position of mocap marker 2 (+y) in meters.
                
            kwargs["mocap_3"] : 3Vector
                The noisy measured position of mocap marker 1 (-x) in meters.
            
            kwargs["mocap_4"] : 3Vector
                The noisy measured position of mocap marker 2 (-y) in meters.
            kwargs["next_gate"] : 3Vector
                The position of the center of the next gate in meters.
            
            kwargs["dir_gate"] : 3Vector
                A direction vector pointing in the direction of the gate.
            
            kwargs["is_last_gate"] : bool
                A Boolean value that indicates whether or not the next gate is the last gate.
            
            kwargs["pos_others"] : List of 3Vectors
                A list of the positions of any other quadrotors in meters.
    
        Returns
        -------
        inputs : List with length equal to the number of inputs
            This is a list of the inputs calculated by the controller. In this
            project, there are four inputs, f_z, tau_x, tau_y, and tau_z.
            Therefore, inputs has the form:
            inputs = [tau_x, tau_y, tau_z, f_z]
        
        """
        # FIXEME: Replace the null controller with your controller
        return [0., 0., 0., 0.]

In [ ]:
# Create an instance of our Controller class to simulate 1 quadrotor
controller1 = Controller()

## Running the Simulation

Before we run the simulation, we must first import the simulation module and then create an instance of the simulator. This is done below:

In [ ]:
# Import the quadrotor project. This module is used to simulate, render, and plot a quadrotor dynamic system
import ae353_quadrotor

There are eight main parameters to the Quadrotor_Sim class initialization function:
#### Parameters

> **n_quadrotors** : *int*  
> The number of quadrotor controllers provided to the simulator.
> 
> **team_list** : *list of strings, shape(n_quadrotors,)*  
> A list of the team names associated with each of the n_quadrotors controllers. The teams names to select from include: "*the ORB*", "*Team Kachow*", "*Team Steam Tunnels*", "*Team Flying Mambas*", "*Team Spare Chang-e*", and "*the Flying Illini*". The string "" results in an empty team flag.
>
> **planet** : *string*,
> The name of the planet on which the race takes place. This parameter varies the acceleration due to graivty. You must select from one of the following strings: "Mercury", "Venus", "Earth", "Moon", "Mars", "Jupiter", "Saturn", "Uranus", "Neptune", "Pluto". Note that this does not alter the maximum thrust of the quadrotor.    
> 
> **use_keyboard** : *bool, optional*  
> A boolean flag that indicates whether the simulation will allow the use of keyboard interactivity. The default is True.
> 
> **visualization** : *bool, optional*  
> A boolean flag that indicates whether the simulation will be  visualized in meshcat. The default is True.
>
> **visualization_fr** : *float, optional*  
> The frame rate (frames per second) at which the visualizer is  updated. The default is 45.  .
> 
> **animation** : *bool, optional*  
> A boolean flag that indicates whether animated plots are created in real time. The default is True.
>
>  **verbose** : *bool, optional*  
> A boolean flag that indicates whether warnings are printing during  function execution. The default is False.

In [ ]:
# Make an instance of the simulator for 1 quadrotor
team_list = [""]
sim = ae353_quadrotor.Quadrotor_Sim(n_quadrotors=1,
                                    team_list=team_list,
                                    planet="Earth",
                                    use_keyboard=True,
                                    visualization=True,
                                    visualization_fr=45.,
                                    animation=False,
                                    verbose=True)

Now we are ready to run the simulation and collect data. To do this we call the ``sim.run()`` function. This function takes a list of members of the  ``Controller`` class as an argument and returns ``track_stats`` and ``data``. The parameters and return values of ``sim.run()`` are shown below.

#### Parameters

> **controllers** : *List of members of Controller class*  
> A list of length n_quadrotors where each element is a member of  a custom class that, at a minimum, must provide the function   controller.run() and controller.reset.()
> 
> **max_time** : *Float or None, optional*  
> The total amount of time the simulation is allowed to run. If set to None, the simulation will run until "ESC" is pressed on the keyboard. If the keyboard is disabled, and max_time is set to None, the simulation will automatically terminate after 10 seconds of simulated time. The default value is None.
>  
> **sensor_noise** : *Float, optional*  
> The noise applied to the mocap sensor, i.e., the standard deviation of the components of each mocap marker tracker measurement. The default value is 0.01 (1 cm).
>
>  **verbose** : *bool, optional*  
> A boolean flag that indicates whether warnings are printing during function execution. The default is False.
> 
#### Returns

> **track_stats** : *List of Dictionaries*
> A list of dictionaries containing all relevant track performance data generated during the simulation. There is one dictionary for each quadrotor simulated. Specific data is extracted via the statement: track_stats[QUADROTOR NUMBER]["KEYWORD"], where QUADROTOR NUMBER is the index of the quadrotor for which track stats are collected and "KEYWORD" is a key to the dictionary. The keys of track_stats for this project are as follows:
>
> **track_stats[QUADROTOR NUMBER]["completed_lap"]** : *Bool*  
> A boolean value that indicates whether or not the quadrotor completed the lap.  
>
> **track_stats[QUADROTOR NUMBER]["gate_times"]** : *List of Floats*  
> The time at which each gate was passed by the quadrotor. The length of this list is equal to the number of gates passed by the quadrotor. If no gates were passed, is an empty list.  
>
> **track_stats[QUADROTOR NUMBER]["lap_time"]** : *Float*  
> The time at which the final gate was passed by the quadrotor. If the final gate was not passed, has the value None.  
>  
> **data** : *List of Dictionary of Lists*  
> A list of dictionaries containing all relevant simulation data generated during the simulation. There is one dictionary for each quadrotor simulated. Specific data is extracted via the statement: data[QUADROTOR NUMBER]["KEYWORD"], where QUADROTOR NUMBER is the index of the quadrotor for which data is collected and "KEYWORD" is a key to the dictionary. The keys of data for this project are as follows:
>  
> **data[QUADROTOR NUMBER]["position"]** : *List of arrays, shape(n,3)*  
> The [x, y, z] position of the quadrotor at each time stamp.  
>  
> **data[QUADROTOR NUMBER]["orientation"]** : *List of arrays, shape(n,3)*  
> The [roll, pitch, yaw] orientation of the quadrotor at each  time stamp.  
>  
> **data[QUADROTOR NUMBER]["velocity"]** : *List of arrays, shape(n,3)*  
>The [vx, vy, vz] velocity of the quadrotor in body fixed  coordinates at each time stamp.  
>  
> **data[QUADROTOR NUMBER]["angular velocity"]** : *List of arrays, shape(n,3)*  
The [wx, wy, wz] angular velocity of the quadrotor in body  fixed coordinates at each time stamp.  
> 
> **data[QUADROTOR NUMBER]["mocap 1"]** : *List of arrays, shape(n,3)*  
> The true (not noisy) position of the mocap 1 marker (+x) in  meters at each time stamp.  
> 
> **data[QUADROTOR NUMBER]["mocap 2"]** : *List of arrays, shape(n,3)*  
> The true (not noisy) position of the mocap 2 marker (+y) in  meters at each time stamp.  
> 
> **data[QUADROTOR NUMBER]["mocap 3"]** : *List of arrays, shape(n,3)*  
> The true (not noisy) position of the mocap 3 marker (-x) in  meters at each time stamp.  
> 
> **data[QUADROTOR NUMBER]["mocap 4"]** : *List of arrays, shape(n,3)*  
> The true (not noisy) position of the mocap 4 marker (-y) in  meters at each time stamp.  
> 
> **data[QUADROTOR NUMBER]["inputs"]** : *List of arrays, shape(n,4)*  
> The inputs applied to the quadrotor. [tau_x, tau_y, tau_z, f_z]  
> 
> **data[QUADROTOR NUMBER]["time"]** : *List of Floats, shape(n,)*  
> A list of the time stamps in seconds. 

#### If enabled you can use the keyboard to interact with the simulation:  
> press **ENTER** to start the simulation.  
> press **BACKSPACE** to reset the simulation.  
> press **SPACE** to pause the simulation.  
> press **ESC** to end the simulation.  

In [ ]:
# Run the simulation and collect the simulation data\
controllers = [controller1]
(track_stats, data) = sim.run(controllers,
                              sensor_noise=0.01,
                              collect_data=True,
                              max_time=60.0,
                              verbose=True)

In [ ]:
# Print out the track stats for each quadrotor
for i in range(len(track_stats)):
    stat = track_stats[i]
    print("Quadrotor {} completed lap: {}".format(i, stat["completed_lap"]))
    if len(stat["gate_times"]) > 0:
        print("Gate times for Quadrotor {}: {}".format(i, stat["gate_times"]))
    else:
        print("Quadrotor {} passed no gates.".format(i))
    if stat["completed_lap"]:
        print("Lap time for Quadrotor {}: {}".format(i, stat["lap_time"]))
    print("\n")

## FIXME: Post-processing the simulation data

After the simulation is run, we can plot the data using the Matplotlib package. To start, we import matplotlib and numpy.

In [ ]:
# Import the pyplot module from the matplotlib package
import matplotlib.pyplot as plt

# Import numpy
import numpy as np

Now we can make whichever plots we see fit to make sense of the simulation data. Here we plot the ground track and altitude.

**THIS IS LEFT INTENTIONALLY AS INSUFFICIENT TO PROVE YOUR CONTROLLER OR OBSERVER WORK.**

In [ ]:
# Create a figure that has two plots in it
fig, axes = plt.subplots(2, 1, figsize=(7.2, 5.4), sharex=False)

# Convert the position data to array
colors = ['k','r','g','b','m','c','tab:orange','tab:purple','tab:brown','tab:pink','tab:gray']
for i in range(len(data)):
    
    # Get the position
    pos = np.array(data[i]['position'])
    
    # Create a plot of the ground path of the quadrotor
    axes[0].plot(pos[:,0], pos[:,1], c=colors[i], lw=2.5, label=str(i+1))
    
    # Create a plot of the altitude versus time
    axes[1].plot(data[0]['time'], pos[:,2], c=colors[i], lw=2.5, label=str(i+1))

# Make plot 0 pretty
axes[0].set_xlabel("X Position [meters]", fontsize=12)
axes[0].set_ylabel("Y Position [meters]", fontsize=12)
axes[0].tick_params(axis='y', labelsize=12)
axes[0].set_title("Ground Track", fontsize=14)
axes[0].set_aspect('equal', 'box')
axes[0].legend(bbox_to_anchor=(1.16, 1.05), fancybox=True, shadow=True)
axes[0].grid()

# Make plot 1 pretty
axes[1].set_xlabel("Time [seconds]", fontsize=12)
axes[1].set_ylabel("Altitude [meters]", fontsize=12)
axes[1].tick_params(axis='y', labelsize=12)
axes[1].set_title("Altitude vs. Time", fontsize=14)
axes[1].legend(bbox_to_anchor=(1., 1.05), fancybox=True, shadow=True)
axes[1].grid()

# Show the figure
fig.tight_layout(pad=2.0)
plt.show()